## Workflows:

1. Update config.yaml
2. Update secrets.yaml [Optional]
3. Update params.yaml
4. Update the entity
5. Update the configuration manager in src config
6. Update the components
7. Update the pipeline 
8. Update the main.py
9. Update the dvc.yaml
10. app.py


In [7]:
cd ../

d:\data_science\kidney_disease_classification


d:\data_science\kidney_disease_classification\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
pwd

'd:\\data_science\\kidney_disease_classification'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen=True)
class ModelTrainingConfig:
  root_dir: Path
  trained_model_path: Path
  updated_base_model_path: Path
  training_data: Path
  params_epochs: int
  params_batch_size: int
  params_is_augmentation: int
  params_image_size: list

In [15]:
# Update the configuration manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories# Placeholder content
from cnnClassifier.entity.config_entity import DataIngestionConfig, PrepareBaseModelConfig
import os

class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_path = PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
    
    def model_training_config(self) -> ModelTrainingConfig:
        training =self.config.model_training
        params = self.params
        create_directories([training.root_dir])
        
        training_config = ModelTrainingConfig(
            root_dir=training.root_dir,
            trained_model_path=training.trained_model_path,
            updated_base_model_path=self.config.prepare_base_model.updated_model_path,
            training_data=os.path.join(self.config.data_ingestion.unzip_dir, 'kidney-ct-scan-image'),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        
        return training_config

In [44]:
# Update the components
import tensorflow as tf
from pathlib import Path
from cnnClassifier.entity.config_entity import ModelTrainingConfig

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_ds(self):
        img_height = self.config.params_image_size[:-1][0]
        img_width = self.config.params_image_size[:-1][1]
        img_size = (img_height, img_width)
        
        self.train_ds = tf.keras.utils.image_dataset_from_directory(
                self.config.training_data,
                validation_split=0.2,
                subset="training",
                seed=123,
                image_size=img_size,
                batch_size=self.config.params_batch_size)
        
        self.val_ds = tf.keras.utils.image_dataset_from_directory(
                self.config.training_data,
                validation_split=0.2,
                subset="validation",
                seed=123,
                image_size=img_size,
                batch_size=self.config.params_batch_size)
        
        # Optimizing
        AUTOTUNE = tf.data.AUTOTUNE

        self.train_ds = self.train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
        self.val_ds = self.val_ds.cache().prefetch(buffer_size=AUTOTUNE)
        
        
        # Normalizing
        normalization_layer = tf.keras.layers.Rescaling(1./255)
        
        self.train_ds = self.train_ds.map(lambda x, y: (normalization_layer(x), y))
        self.val_ds = self.val_ds.map(lambda x, y: (normalization_layer(x), y))
        
        # Augmenting train dataset
        data_augmentation = tf.keras.Sequential(
            [
                tf.keras.layers.RandomFlip("horizontal",
                                input_shape=(img_height,
                                            img_width,
                                            3)),
                tf.keras.layers.RandomRotation(0.1),
                tf.keras.layers.RandomZoom(0.1),
            ]
            )
        self.train_ds = self.train_ds.map(lambda x, y: (data_augmentation(x), y))             
        
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        
        self.model.summary()
        self.model.fit(
            self.train_ds,
            epochs=self.config.params_epochs,
            validation_data=self.val_ds
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [45]:
# Updating pipeline
from cnnClassifier.utils import logger
try:
    config = ConfigurationManager()
    model_training_config = config.model_training_config()
    model = ModelTraining(model_training_config)
    model.get_base_model()
    model.train_valid_ds()
    model.train()
    logger.info(f"Training pipeline succesful!")
except Exception as e:
    raise e

[2023-12-19 15:04:30,964: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-19 15:04:30,965: INFO: common: yaml file: params.yaml loaded successfully]
Found 465 files belonging to 2 classes.
Using 372 files for training.
Found 465 files belonging to 2 classes.
Using 93 files for validation.
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2